In [21]:
filename="input-ex.txt"
with open(filename) as f:
    lines = [line.rstrip().split(":") for line in f.readlines()]
    for i in range(len(lines)):
        line = lines[i]
        line[0] = line[0][10:]
        x_y = line[0].replace("x=","").replace("y=","").split(", ")
        line[0] = tuple(map(int,x_y))
        line[1] = tuple(map(int,line[1][22:].replace("x=","").replace("y=","").split(", ")))

sensors = [sensor[0] for sensor in lines]
beacons = [beacon[1] for beacon in lines]

def print_map(sensors,beacons):
    combined = sensors+beacons
    min_x = min(combined, key=lambda tup: tup[0])[0]
    max_x = max(combined, key=lambda tup: tup[0])[0]
    min_y = min(combined, key=lambda tup: tup[1])[1]
    max_y = max(combined, key=lambda tup: tup[1])[1]

    for y in range(min_y,max_y):
        row_string = ""
        for x in range(min_x,max_x):
            if (x,y) in sensors:
                row_string += "S"
            elif (x,y) in beacons:
                row_string += "B"
            else:
                row_string += "."
        print(row_string)

print("Map: ")
print_map(sensors,beacons)

[[(2, 18), (-2, 15)], [(9, 16), (10, 16)], [(13, 2), (15, 3)], [(12, 14), (10, 16)], [(10, 20), (10, 16)], [(14, 17), (10, 16)], [(8, 7), (2, 10)], [(2, 0), (2, 10)], [(0, 11), (2, 10)], [(20, 14), (25, 17)], [(17, 20), (21, 22)], [(16, 7), (15, 3)], [(14, 3), (15, 3)], [(20, 1), (15, 3)]]
[(2, 18), (9, 16), (13, 2), (12, 14), (10, 20), (14, 17), (8, 7), (2, 0), (0, 11), (20, 14), (17, 20), (16, 7), (14, 3), (20, 1)]
[(-2, 15), (10, 16), (15, 3), (10, 16), (10, 16), (10, 16), (2, 10), (2, 10), (2, 10), (25, 17), (21, 22), (15, 3), (15, 3), (15, 3)]
Map: 
....S......................
......................S....
...............S...........
................SB.........
...........................
...........................
...........................
..........S.......S........
...........................
...........................
....B......................
..S........................
...........................
...........................
..............S.......S....
B..................